In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib qt
#Función para calcular la formación de imgenes de una lente en el plano focal si el objeto está en el otro plano focal
def Lenses2F(image,landa=1,f=1,shift=0):
  #image: Imagen que se ubica en el foco del plano objeto y pasará por la lente 
  #landa: Longitud de onda de la luente de luz que ingresa al sistema
  #f: distancia focal de la lente
  #shift: 0 si no se desea realizar un fftshit a la matriz y 1 en caso contrario
  if (shift==1):
    result = (-1j/(landa*f))*(np.fft.fftshift(np.fft.fftn(image)))
  elif (shift==0):
    result = (-1j/(landa*f))*(np.fft.fftn(image))
  return result
  
#Función para gráficar matrices complejas 
def Complex_Plot(matrix,kind,log,axs,fig = 0, colbar = False):
  #matrix: matriz a gráficar
  #kind: I=Intensidad, A=Amplitud, P=Fase
  #log: 1 si se desea escala logarítmica 0 en caso contrario
  if (kind=='I'):
    matrix_to_plot=np.abs(matrix)**2
  elif (kind=='A'):
    matrix_to_plot=np.abs(matrix)
  elif (kind=='P'):
     matrix_to_plot=np.angle(matrix)
  if (log==1):
    image = axs.imshow(np.log(matrix_to_plot + 0.0000001))
  elif (log==0):
    image = axs.imshow(matrix_to_plot)
  if (colbar):
    fig.colorbar(image, ax = axs)
  return 

#Definimos el sistema 4F a partir de las funciones que ya teniamos definidas.
def Lenses4F(image,landa,f):
    Half01=Lenses2F(image,landa,f,1) #Usamos el método de dividir el sistema 4f en dos mitades.
    Half02=Lenses2F(Half01,landa,f,0)
    return Half02


In [44]:
def Complex_Plot(matrix,kind,log):
  #kind: I=Intensidad, A=Amplitud, P=Fase
  plt.figure()
  if (kind=='I'):
    matrix_to_plot=np.abs(matrix)**2
  elif (kind=='A'):
    matrix_to_plot=np.abs(matrix)
  elif (kind=='P'):
     matrix_to_plot=np.angle(matrix)
  if (log==1):
    plt.imshow(np.log(matrix_to_plot+0.00001))
    plt.colorbar()
  elif (log==0):
    plt.imshow(matrix_to_plot)
    #plt.colorbar()
  return 

In [3]:


#Función para crear regillas circulares
def Mask_Circle(N,r,kind='A'):
  #N dimensión de la matriz
  #r es el radio del circulo (en píxeles)
  #kind: 'A' si es una apertura, 'O' si es un obstáculo
  if (kind=='A'):
    circ_aperture=np.zeros((N,N),dtype="uint8")
    coordy=int(np.shape(circ_aperture)[0]/2)
    coordx=int(np.shape(circ_aperture)[1]/2)
    cv2.circle(circ_aperture,(coordx,coordy),r,1,-1)
    return circ_aperture
  elif (kind=='O'):
    circ_obstacle=np.ones((N,N),dtype="uint8")
    coordy=int(np.shape(circ_obstacle)[0]/2)
    coordx=int(np.shape(circ_obstacle)[1]/2)
    cv2.circle(circ_obstacle,(coordx,coordy),r,0,-1)
    return circ_obstacle
#Función para gráficar matrices complejas 

In [4]:
gala = cv2.imread('gala.jpg', 0)

In [5]:
Complex_Plot(gala,'A',0)

In [6]:
np.shape(gala)


(2835, 2182)

In [31]:
dx=dy=1.48*mm
dfx=1/(2182*dx)
dfy=1/(2835*dy)
mask=np.zeros(np.shape(gala),dtype="uint8")
coordy=int(np.shape(mask)[0]/2)
coordx=int(np.shape(mask)[1]/2)
#cv2.circle(mask,(coordx,coordy),4,1,-1)
cv2.ellipse(mask, (coordx,coordy), (18,30), 0, 0, 360, 1, -1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [32]:
Complex_Plot(mask,'A',0)

In [9]:
gala_fft=(dx**2)*np.fft.fftshift(np.fft.fftn(gala))
gala_filtered=gala_fft*mask
gala_final=(dfx*dfy)*np.fft.ifftn(gala_filtered)

In [10]:
Complex_Plot(gala_final,'A',0)

In [11]:
#####################################################################################

In [12]:
w_length=633*nm

In [13]:
mask_pad=np.zeros((2834,2834),dtype="uint8")
coordy=int(np.shape(mask_pad)[0]/2)
coordx=int(np.shape(mask_pad)[1]/2)
cv2.circle(mask_pad,(coordx,coordy),18,1,-1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [14]:
Complex_Plot(mask_pad,'A',0)

In [15]:
""" Función de Difracción Discreta por Transformada de Fresnel usando FFTs"""
def Fresnel_Transform_FFT(matrix,z,w_length,dx0):
    # matrix es la representación discreta de la imagen o función 
    # z es la distancia de propagación (metros)
    # w_length es la longitud de onda (m)
    # dx0 es el intervalo de muestreo de la función en el plano de entrada

    #Se define la dimensión de la imagen (solo se utiliza N por ser cuadrada)
    N=np.shape(matrix)[0]

    #Se calcula dx, el intervalo de muestreo en el plano de salida
    dx=(w_length*z)/(dx0*N)

    #Se definen las coordenadas del plano de entrada con el fin de aplicar las fases esféricas de entrada
    k=(2*np.pi)/w_length
 

    #Se aplican las fases esféricas de entrada
    U_prima=matrix

    #Se calcula la transformada de Fourier de la función imediatamente anterior
    U_2prima= np.fft.fftshift(np.fft.fftn(U_prima))
    U_2prima=(dx**2)*U_2prima

    #Se definen las coordenadas del plano de salida con el fin de aplicar las fases esféricas de salida
    x=np.arange(-int(N/2),int(N/2),1)
    y=np.arange(-int(N/2),int(N/2),1)
    n,m=np.meshgrid(x,y)

    #Se aplican las fases esféricas de salida y se obtiene el campo difractado a un distancia z
    U_z=U_2prima*(np.exp(1j*k*z)/(1j*w_length*z))*np.exp((1j*k/(2*z))*((dx*n)**2+(dx*m)**2))
    return U_z

In [ ]:
zi=20*mm
zo=168
M=-zi/zo

In [ ]:
h=Fresnel_Transform_FFT(mask_pad,20*mm,w_length,dfx)
h=(1/abs(M))*h

In [ ]:
Complex_Plot(h,'A',0)

In [ ]:
Ug=(1/abs(M))*gala
lincoln=np.fft.ifftn((dx**2)*(np.fft.fftshift(np.fft.fftn(Ug)))*mask)

In [ ]:
Complex_Plot(lincoln,'A',0)

In [ ]:
Complex_Plot(gala,'A',0)

In [19]:
N=1000
nm=1e-9
mm=1e-3
dx=0.01*mm
w_length=2*mm
x=np.arange(-int(N/2),int(N/2),1)
y=np.arange(-int(N/2),int(N/2),1)
X,Y=np.meshgrid(x,y)

k=2*np.pi/w_length


img=np.cos(X*dx*k)+1


In [20]:
Complex_Plot(img,'A',0)

In [22]:
Complex_Plot((dx*2)*np.fft.fftshift(np.fft.fftn(img)),'A',1)

# Punto 1

In [48]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

#Definimos unidades
nm = 1e-9
um = 1e-6
mm = 1e-3
m = 1

#Función para crear rejillas circulares con diferentes coordenadas y tamaños
def Mask_Circle(shape,r,kind='A',coordx=0,coordy=0):
    #shape dimensiones de la matriz (2-tupla)
    #r es el radio del circulo (en píxeles)
    #kind: 'A' si es una apertura, 'O' si es un obstáculo
    #coordx y coordy son las coordenadas del centro en el eje x y y respectivamente. Definidas en el centro de la imagen por defecto
    if (coordx==0):
        coordx=int(shape[1]/2)
    if (coordy==0):
        coordy=int(shape[0]/2)

    if (kind=='A'):
        circ_aperture=np.zeros(shape,dtype="uint8")
        cv2.circle(circ_aperture,(coordx,coordy),r,1,-1)
        return circ_aperture
    elif (kind=='O'):
        circ_obstacle=np.ones(shape,dtype="uint8")
        cv2.circle(circ_obstacle,(coordx,coordy),r,0,-1)
        return circ_obstacle

#Función para gráficar matrices complejas 
def Complex_Plot(matrix,kind,log,axs,fig = 0, colbar = False):
  #matrix: matriz a gráficar
  #kind: I=Intensidad, A=Amplitud, P=Fase
  #log: 1 si se desea escala logarítmica 0 en caso contrario

  if (kind=='I'):
    matrix_to_plot=np.abs(matrix)**2
  elif (kind=='A'):
    matrix_to_plot=np.abs(matrix)
  elif (kind=='P'):
     matrix_to_plot=np.angle(matrix)
  if (log==1):
    image = axs.imshow(np.log(matrix_to_plot + 0.0000001))
  elif (log==0):
    image = axs.imshow(matrix_to_plot)
  if (colbar):
    fig.colorbar(image, ax = axs)
  return 

  #Función para crear transmitancias con forma de elipse
def Mask_Ellipse(shape,rx,ry,kind='A'):
  #shape son las dimensiones de la matriz (2-tupla)
  #rx es el radio de la elipse en la dirección horizontal
  #ry es el radio de la elipse en la dirección vertical
  #kind: 'A' si es una apertura, 'O' si es un obstáculo
  if (kind=='A'):
    e_aperture=np.zeros(shape,dtype="uint8")
    coordy=int(np.shape(e_aperture)[0]/2)
    coordx=int(np.shape(e_aperture)[1]/2)
    cv2.ellipse(e_aperture, (coordx,coordy), (rx,ry), 0, 0, 360, 1, -1)
    return e_aperture
  elif (kind=='O'):
    e_obstacle=np.ones(shape,dtype="uint8")
    coordy=int(np.shape(e_obstacle)[0]/2)
    coordx=int(np.shape(e_obstacle)[1]/2)
    cv2.ellipse(e_obstacle, (coordx,coordy), (rx,ry), 0, 0, 360, 0, -1)
    return e_obstacle

In [58]:
%matplotlib qt
plt.set_cmap('gist_gray')

##############################################################
#Se importa la imagen del lienzo
#Definimos los parametros y la imagen 
root = "resources/"
filename = "gala.jpg"
gala = cv2.imread('gala.jpg', 0)

#Se grafica el lienzo para determinar el tamaño de los macropixeles
fig1, axs = plt.subplots(1, 2, sharey=False)
Complex_Plot(gala,'A',0,axs[0],fig1)
axs[0].set_title('(a)')
axs[0].set_xlabel('1.48mm/píxel')
Complex_Plot(gala,'A',0,axs[1],fig1)
axs[1].set_title('(b)')
axs[1].set_xlabel('1.48mm/píxel')


#Se definen los tamaños de píxel de la imagen y del Plano de Fourier
dx=dy=1.48*mm
dfx=1/(np.shape(gala)[1]*dx)
dfy=1/(np.shape(gala)[0]*dy)

#Se define la resolucion que se desea obtener y se calculan los radios de la transmitancia que filtrará las frecuencias
#espaciales en el Plano de Fourier
resolution=237*mm
rx=int(1/(resolution*dfx))
ry=int(1/(resolution*dfy))

#Se genera la transmitancia en forma de elipse
mask=Mask_Ellipse(np.shape(gala),rx,ry,kind='A')

#Se grafica la transmitancia
fig2, axs = plt.subplots(1, 1)
Complex_Plot(mask,'A',0,axs,fig2)

#Se hace la transformada de fourier del lienzo, se multiplica por la transmitancia y luego se hace la transformada inversa
#para obtener la imagen con la resolución definida previamente
gala_fft=(dx**2)*np.fft.fftshift(np.fft.fftn(gala))
gala_filtered=gala_fft*mask
gala_final=(dfx*dfy)*np.fft.ifftn(gala_filtered)

fig3, axs = plt.subplots(1, 1)
Complex_Plot(gala_final,'A',0,axs,fig3)

# A continuación se hará el filtrado espacial de frecuencias simulando diferentes distancias de alejamiento, la resolución
# en cada distancia se calcula teniendo en cuenta que la resolución del ojo humano es de 5 min de arco

def Resolution(distance):
    return 2*distance*np.tan(0.000727)

def Gala_At_Distances(img, rang,dx,dy):
    dfx=1/(np.shape(img)[1]*dx)
    dfy=1/(np.shape(img)[0]*dy)
    images=[]
    for i in rang:
        resolution=Resolution(i)
        rx=int(1/(resolution*dfx))
        ry=int(1/(resolution*dfy))
        mask=Mask_Ellipse(np.shape(img),rx,ry,kind='A')
        img_fft=(dx**2)*np.fft.fftshift(np.fft.fftn(img))
        img_filtered=img_fft*mask
        img_final=(dfx*dfy)*np.fft.ifftn(img_filtered)
        
        plt.figure()
        plt.imshow(np.abs(img_final))
        plt.title(f'{i} metros de alejamiento')
        
    return 
        
#Gala_At_Distances(gala,range(180,10,-10),dx,dy) #Descomentar esta línea de código para generar 17 imágenes con 
                                                #pérdida de resolución progresiva desde los 20m a 180m con saltos de 10m





C:\Users\ezque\AppData\Local\Temp/ipykernel_11552/881990615.py:67: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


In [59]:
# A continuación se hará el filtrado espacial de frecuencias simulando diferentes distancias de alejamiento, la resolución
# en cada distancia se calcula teniendo en cuenta que la resolución del ojo humano es de 5 min de arco

def Resolution(distance):
    return 2*distance*np.tan(0.000727)

def Gala_At_Distances(img, rang,dx,dy):
    dfx=1/(np.shape(img)[1]*dx)
    dfy=1/(np.shape(img)[0]*dy)
    a=0
    c=['(a)','(b)']
    fig4, axs = plt.subplots(1, 2,sharey=True)
    for i in rang:
        resolution=Resolution(i)
        rx=int(1/(resolution*dfx))
        ry=int(1/(resolution*dfy))
        mask=Mask_Ellipse(np.shape(img),rx,ry,kind='A')
        img_fft=(dx**2)*np.fft.fftshift(np.fft.fftn(img))
        img_filtered=img_fft*mask
        img_final=(dfx*dfy)*np.fft.ifftn(img_filtered)

        Complex_Plot(img_final,'A',0,axs[a],fig4)
        axs[a].set_title(c[a])
        axs[a].set_xlabel('{} metros de distancia'.format(i))
        a=a+1

Gala_At_Distances(gala,range(100,120,10),dx,dy)

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib qt

#Definimos unidades
nm = 1e-9
um = 1e-6
mm = 1e-3
m = 1

#Función para crear regillas circulares
def Mask_Ellipse(shape,rx,ry,kind='A'):
  #N dimensión de la matriz
  #r es el radio del circulo (en píxeles)
  #kind: 'A' si es una apertura, 'O' si es un obstáculo
  if (kind=='A'):
    e_aperture=np.zeros(shape,dtype="uint8")
    coordy=int(np.shape(e_aperture)[0]/2)
    coordx=int(np.shape(e_aperture)[1]/2)
    cv2.ellipse(e_aperture, (coordx,coordy), (rx,ry), 0, 0, 360, 1, -1)
    return e_aperture
  elif (kind=='O'):
    e_obstacle=np.ones(shape,dtype="uint8")
    coordy=int(np.shape(e_obstacle)[0]/2)
    coordx=int(np.shape(e_obstacle)[1]/2)
    cv2.ellipse(e_obstacle, (coordx,coordy), (rx,ry), 0, 0, 360, 0, -1)
    return e_obstacle
#Función para gráficar matrices complejas 
#cv2.ellipse(mask, (coordx,coordy), (18,30), 0, 0, 360, 1, -1)
def Complex_Plot(matrix,kind,log):
  #kind: I=Intensidad, A=Amplitud, P=Fase
  plt.figure()
  plt.set_cmap('gist_gray')  
  if (kind=='I'):
    matrix_to_plot=np.abs(matrix)**2
  elif (kind=='A'):
    matrix_to_plot=np.abs(matrix)
  elif (kind=='P'):
     matrix_to_plot=np.angle(matrix)
  if (log==1):
    plt.imshow(np.log(matrix_to_plot+0.00001))
    plt.colorbar()
  elif (log==0):
    plt.imshow(matrix_to_plot)
    #plt.colorbar()
    
 
  return 

In [7]:
gala = cv2.imread('gala.jpg', 0)
Complex_Plot(gala,'A',0)

#Primero se hará el estudio de filtrar frecuencias mayores a las correspondientes al tamaño de los macropixeles 
#(aproximadamente 237mm x 237mm)

dx=dy=1.48*mm
dfx=1/(np.shape(gala)[1]*dx)
dfy=1/(np.shape(gala)[0]*dy)

resolution=237*mm
rx=int(1/(resolution*dfx))
ry=int(1/(resolution*dfy))

mask=Mask_Ellipse(np.shape(gala),rx,ry,kind='A')

Complex_Plot(mask,'A',0)

gala_fft=(dx**2)*np.fft.fftshift(np.fft.fftn(gala))
gala_filtered=gala_fft*mask
gala_final=(dfx*dfy)*np.fft.ifftn(gala_filtered)

Complex_Plot(gala_final,'A',0)



In [25]:
# A continuación se hará el filtrado espacial de frecuencias simulando diferentes distancias de alejamiento, la resolución
# en cada distancia se calcula teniendo en cuenta que la resolución del ojo humano es de 5 min de arco

def Resolution(distance):
    return 2*distance*np.tan(0.000727)

def Gala_At_Distances(img, rang,dx,dy):
    dfx=1/(np.shape(img)[1]*dx)
    dfy=1/(np.shape(img)[0]*dy)
    for i in rang:
        resolution=Resolution(i)
        rx=int(1/(resolution*dfx))
        ry=int(1/(resolution*dfy))
        mask=Mask_Ellipse(np.shape(img),rx,ry,kind='A')
        img_fft=(dx**2)*np.fft.fftshift(np.fft.fftn(img))
        img_filtered=img_fft*mask
        img_final=(dfx*dfy)*np.fft.ifftn(img_filtered)

        Complex_Plot(img_final,'A',0)
        plt.title('Imagen {}'.format(int(i/10)-1))

In [26]:
Gala_At_Distances(gala,range(180,10,-10),1.48*mm,1.48*mm)

TypeError: Complex_Plot() missing 1 required positional argument: 'axs'

In [ ]:
def Gala_At_Radius(img, rang,dx,dy):
    dfx=1/(np.shape(img)[1]*dx)
    dfy=1/(np.shape(img)[0]*dy)
    for i in rang:
        #resolution=Resolution(i)
        rx=ry=i
        mask=Mask_Ellipse(np.shape(img),rx,ry,kind='A')
        img_fft=(dx**2)*np.fft.fftshift(np.fft.fftn(img))
        img_filtered=img_fft*mask
        img_final=(dfx*dfy)*np.fft.ifftn(img_filtered)
        Complex_Plot(img_final,'A',0)
        

In [70]:
Gala_At_Radius(gala,range(1000,0,-100),1.48*mm,1.48*mm)